In [ ]:
import os
import sys

OPENAI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

loader=TextLoader('analects_raw.txt',encoding='utf-8')
data=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=10)
splits=text_splitter.split_documents(data)
embedding=OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
vectordb=Chroma.from_documents(
    documents=splits,embedding=embedding,collection_name='personal-data'
)
llm=ChatOpenAI(temperature=0, model="gpt-4")
personal_data=RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',retriever=vectordb.as_retriever()
)

In [ ]:
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor

personal_data_tool=Tool(
    name='PersonalData',
    func=personal_data.run,
    description='논어 데이터'
)
memory_key='history'
memory=ConversationBufferMemory(memory_key=memory_key,return_messages=True,input_key="input",output_key="output")

system_message=SystemMessage(
    content=(
        """
        항상 논어 데이터만을 활용해서 논어랑 가장 관련성 높은 대답을 해줘
        너가 말한 내용의 근거를 내가 준 논어 데이터에서 찾아서 그대로 말해줘
        """
    )
)
prompt=OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
)
tools=[personal_data_tool]
agent=OpenAIFunctionsAgent(llm=llm,tools=tools,prompt=prompt)
agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True, #Added
)

In [ ]:
from langchain.prompts import  PromptTemplate
import pickle

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

with open("conf_styles.pickle", "rb") as pr:
    examples = pickle.load(pr)

In [ ]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, OpenAIEmbeddings(), Chroma, k=2
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=
    """
너는 사용자의 문장을 공자의 말투로 바꿔주는 인공지능이야.
사용자의 질문을 공자라면 어떻게 말했을지 생각해서 같은 의미의 문장을 답변해줘.
반드시 공자의 말투로 번호식으로 배열보다는 줄글로 답변해줘.
공자가 말하듯이 공자의 1인칭으로 말해줘
""",
    suffix="Input: {input_noun}\n",
    input_variables=["input_noun"],
)

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0, model="gpt-4")
chain = LLMChain(llm=chat_model, prompt=similar_prompt, verbose=True)


In [ ]:
input_text = '효도란 무엇인가'
result = agent_executor({'input':input_text})
conf_result = chain(similar_prompt.format(input_noun=result['output']))
print(conf_result["text"])


In [ ]:
#chain for comparison
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

prompt_2 = PromptTemplate(input_variables=["in"],
template="""
효란 무엇인가.
input:{in}
""")
chat_model = ChatOpenAI(temperature=0, model="gpt-4")
chain_b = LLMChain(llm=chat_model,prompt=prompt_2, verbose=True)

input_text ="""adskfhasfl"""
conf_result_b = chain_b(input_text)
print(conf_result_b["text"])

In [ ]:
#Evaluation-Agent Trajectory 
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("trajectory")
evaluation_result = evaluator.evaluate_agent_trajectory(
    prediction=result["output"],
    input=result["input"],
    agent_trajectory=result["intermediate_steps"],
)
evaluation_result

In [ ]:
#Evaluation-correctness criterion
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("labeled_criteria", criteria="correctness")
eval_result = evaluator.evaluate_strings(
    prediction=conf_result["text"],
    reference="",
    input=input_text,
)
print(eval_result)

In [ ]:
#Evaluation-custom criterion
from langchain.evaluation import load_evaluator

custom_criterion = {
    "historic": "Does the output contains ancient wisdom?"
    
}
evaluator_custom =  load_evaluator(
    #"criteria",
    "score_string",
    criteria = custom_criterion,
)
eval_result = evaluator_custom.evaluate_strings(
    #prediction=conf_result["text"],
    prediction=conf_result_b["text"],
    input=input_text,
)
print(eval_result)